In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc gem-suite matplotlib

*Estimativa de uso: 3 minutos em um processador Heron r2 (NOTA: Esta é apenas uma estimativa. Seu tempo de execução pode variar.)*

## Contexto

Este tutorial demonstra como realizar uma transição de fase de Nishimori em um processador quântico IBM&reg;. Este experimento foi originalmente descrito em [*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863).

A transição de fase de Nishimori refere-se à transição entre fases ordenadas de curto e longo alcance no modelo de Ising com ligações aleatórias. Em um computador quântico, a fase ordenada de longo alcance se manifesta como um estado no qual os qubits estão emaranhados por todo o dispositivo. Este estado altamente emaranhado é preparado usando o protocolo de *geração de emaranhamento por medição* (GEM, do inglês *generation of entanglement by measurement*). Ao utilizar medições intermediárias, o protocolo GEM é capaz de emaranhar qubits por todo o dispositivo usando circuitos de apenas profundidade constante. Este tutorial usa a implementação do protocolo GEM do pacote de software [GEM Suite](https://github.com/qiskit-community/gem-suite).

## Requisitos

Antes de começar este tutorial, certifique-se de ter o seguinte instalado:

- Qiskit SDK v1.0 ou posterior, com suporte para [visualização](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 ou posterior ( `pip install qiskit-ibm-runtime` )
- GEM Suite ( `pip install gem-suite` )

## Configuração

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## Passo 1: Mapear entradas clássicas para um problema quântico
O protocolo GEM funciona em um processador quântico com conectividade de qubits descrita por uma rede. Os processadores quânticos da IBM de hoje usam a [rede heavy hex](https://www.ibm.com/quantum/blog/heavy-hex-lattice). Os qubits do processador são agrupados em *plaquetas* com base em qual célula unitária da rede eles ocupam. Como um qubit pode ocorrer em mais de uma célula unitária, as plaquetas não são disjuntas. Na rede heavy hex, uma plaqueta contém 12 qubits. As próprias plaquetas também formam uma rede, onde duas plaquetas estão conectadas se compartilham quaisquer qubits. Na rede heavy hex, plaquetas vizinhas compartilham 3 qubits.

No pacote de software GEM Suite, a classe fundamental para implementar o protocolo GEM é `PlaquetteLattice`, que representa a rede de plaquetas (que é distinta da rede heavy hex). Uma `PlaquetteLattice` pode ser inicializada a partir de um mapa de acoplamento de qubits. Atualmente, apenas mapas de acoplamento heavy hex são suportados.

A célula de código a seguir inicializa uma rede de plaquetas a partir do mapa de acoplamento de um processador quântico da IBM. A rede de plaquetas nem sempre engloba todo o hardware. Por exemplo, `ibm_torino` tem 133 qubits no total, mas a maior rede de plaquetas que cabe no dispositivo usa apenas 125 deles e compreende um total de 18 plaquetas. Algo similar pode ser observado para dispositivos IBM Quantum&reg; com contagens de qubits diferentes também.

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

Você pode visualizar a rede de plaquetas gerando um diagrama de sua representação em grafo. No diagrama, as plaquetas são representadas por hexágonos rotulados, e duas plaquetas estão conectadas por uma aresta se compartilham qubits.

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

Você pode recuperar informações sobre plaquetas individuais, como os qubits que elas contêm, usando o método `plaquettes`.

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

Você também pode produzir um diagrama dos qubits subjacentes que formam a rede de plaquetas.

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

Além dos rótulos de qubits e das arestas indicando quais qubits estão conectados, o diagrama contém três informações adicionais que são relevantes para o protocolo GEM:
- Cada qubit está sombreado (cinza) ou não sombreado. Os qubits sombreados são qubits de "sítio" que representam os sítios do modelo de Ising, e os qubits não sombreados são qubits de "ligação" usados para mediar interações entre os qubits de sítio.
- Cada qubit de sítio é rotulado como (A) ou (B), indicando um de dois papéis que um qubit de sítio pode desempenhar no protocolo GEM (os papéis são explicados posteriormente).
- Cada aresta é colorida usando uma de seis cores, particionando assim as arestas em seis grupos. Esse particionamento determina como portas de dois qubits podem ser paralelizadas, bem como diferentes padrões de escalonamento que provavelmente incorrerão em diferentes quantidades de erro em um processador quântico ruidoso. Como as arestas em um grupo são disjuntas, uma camada de portas de dois qubits pode ser aplicada nessas arestas simultaneamente. Na verdade, é possível particionar as seis cores em três grupos de duas cores de tal forma que a união de cada grupo de duas cores ainda seja disjunta. Portanto, apenas três camadas de portas de dois qubits são necessárias para ativar cada aresta. Existem 12 maneiras de particionar as seis cores dessa forma, e cada partição produz um cronograma de portas de 3 camadas diferente.

Agora que você criou uma rede de plaquetas, o próximo passo é inicializar um objeto `GemExperiment`, passando tanto a rede de plaquetas quanto o backend no qual você pretende executar o experimento. A classe `GemExperiment` gerencia a implementação real do protocolo GEM, incluindo a geração de circuitos, envio de trabalhos e análise dos dados. A célula de código a seguir inicializa a classe de experimento enquanto restringe a rede de plaquetas a apenas duas das plaquetas (21 qubits), reduzindo o tamanho do experimento para garantir que o ruído no hardware não sobrecarregue o sinal.

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

Um circuito do protocolo GEM é construído usando os seguintes passos:
1. Preparar o estado todo-$|+\rangle$ aplicando uma porta de Hadamard a cada qubit.
2. Aplicar uma porta $R_{ZZ}$ entre cada par de qubits conectados. Isso pode ser alcançado usando 3 camadas de portas. Cada porta $R_{ZZ}$ age em um qubit de sítio e um qubit de ligação. Se o qubit de sítio for rotulado (B), então o ângulo é fixado em $\frac{\pi}{2}$. Se o qubit de sítio for rotulado (A), então o ângulo pode variar, produzindo circuitos diferentes. Por padrão, o intervalo de ângulos é definido como 21 pontos igualmente espaçados entre $0$ e $\frac{\pi}{2}$, inclusive.
3. Medir cada qubit de ligação na base de Pauli $X$. Como os qubits são medidos na base de Pauli $Z$, isso pode ser realizado aplicando uma porta de Hadamard antes de medir o qubit.

Observe que o artigo citado na introdução deste tutorial usa uma convenção diferente para o ângulo $R_{ZZ}$, que difere da convenção usada neste tutorial por um fator de 2.

No passo 3, apenas os qubits de ligação são medidos. Para entender em que estado os qubits de sítio permanecem, é instrutivo considerar o caso em que o ângulo $R_{ZZ}$ aplicado aos qubits de sítio (A) no passo 2 é igual a $\frac{\pi}{2}$. Neste caso, os qubits de sítio são deixados em um estado altamente emaranhado semelhante ao estado GHZ,

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

Devido à aleatoriedade nos resultados de medição, o estado real dos qubits de sítio pode ser um estado diferente com ordem de longo alcance, por exemplo, $\lvert 00110 \rangle + \lvert 11001 \rangle$. No entanto, o estado GHZ pode ser recuperado aplicando uma operação de decodificação baseada nos resultados de medição. Quando o ângulo $R_{ZZ}$ é ajustado para baixo a partir de $\frac{\pi}{2}$, a ordem de longo alcance ainda pode ser recuperada até um ângulo crítico, que na ausência de ruído, é aproximadamente $0.3 \pi$. Abaixo deste ângulo, o estado resultante não mais exibe emaranhamento de longo alcance. Esta transição entre a presença e ausência de ordem de longo alcance é a transição de fase de Nishimori.

Na descrição acima, os qubits de sítio foram deixados sem medir, e a operação de decodificação pode ser realizada aplicando portas quânticas. No experimento conforme implementado na suíte GEM, que este tutorial segue, os qubits de sítio são de fato medidos, e a operação de decodificação é aplicada em uma etapa de pós-processamento clássico.

Na descrição acima, a operação de decodificação pode ser realizada aplicando portas quânticas aos qubits de sítio para recuperar o estado quântico. No entanto, se o objetivo é medir imediatamente o estado, por exemplo, para fins de caracterização, então os qubits de sítio são medidos junto com os qubits de ligação, e a operação de decodificação pode ser aplicada em uma etapa de pós-processamento clássico. É assim que o experimento é implementado na suíte GEM, que este tutorial segue.

Além de depender do ângulo $R_{ZZ}$ no passo 2, que por padrão varre 21 valores, o circuito do protocolo GEM também depende do padrão de escalonamento usado para implementar as 3 camadas de portas $R_{ZZ}$. Como discutido anteriormente, existem 12 desses padrões de escalonamento. Portanto, o número total de circuitos no experimento é $21 \times 12 = 252$.

Os circuitos do experimento podem ser gerados usando o método `circuits` da classe `GemExperiment`.

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

Para os propósitos deste tutorial, é suficiente considerar apenas um único padrão de escalonamento. A célula de código a seguir restringe o experimento ao primeiro padrão de escalonamento. Como resultado, o experimento tem apenas 21 circuitos, um para cada ângulo $R_{ZZ}$ varrido.

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

A célula de código a seguir desenha um diagrama do circuito no índice 5. Para reduzir o tamanho do diagrama, as portas de medição no final do circuito são removidas.

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## Passo 2: Otimizar problema para execução em hardware quântico
Transpilar circuitos quânticos para execução em hardware normalmente envolve [várias etapas](/guides/transpiler-stages). Normalmente, as etapas que incorrem em maior sobrecarga computacional são a escolha do layout de qubits, o roteamento das portas de dois qubits para se conformar à conectividade de qubits do hardware e a otimização do circuito para minimizar sua contagem de portas e profundidade. No protocolo GEM, as etapas de layout e roteamento são desnecessárias porque a conectividade do hardware já está incorporada no design do protocolo. Os circuitos já têm um layout de qubits, e as portas de dois qubits já estão mapeadas em conexões nativas. Além disso, para preservar a estrutura do circuito conforme o ângulo $R_{ZZ}$ é variado, apenas otimização de circuito muito básica deve ser realizada.

A classe `GemExperiment` transpila circuitos de forma transparente ao executar o experimento. As etapas de layout e roteamento já são substituídas por padrão para não fazer nada, e a otimização de circuito é realizada em um nível que otimiza apenas portas de qubit único. No entanto, você pode substituir ou passar opções adicionais usando o método `set_transpile_options`. Para fins de visualização, a célula de código a seguir transpila manualmente o circuito exibido anteriormente e desenha o circuito transpilado.

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## Passo 3: Executar usando primitivas Qiskit
Para executar os circuitos do protocolo GEM no hardware, chame o método `run` do objeto `GemExperiment`. Você pode especificar o número de disparos que deseja amostrar de cada circuito. O método `run` retorna um objeto [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) que você deve salvar em uma variável. Observe que o método `run` apenas envia trabalhos sem esperar que eles terminem, então é uma chamada não bloqueante.

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

Para aguardar os resultados, chame o método `block_for_results` do objeto `ExperimentData`. Esta chamada fará com que o interpretador aguarde até que os trabalhos sejam concluídos.

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## Passo 4: Pós-processar e retornar resultado em formato clássico desejado
Em um ângulo $R_{ZZ}$ de $\frac{\pi}{2}$, o estado decodificado seria o estado GHZ na ausência de ruído. A ordem de longo alcance do estado GHZ pode ser visualizada plotando a magnetização das cadeias de bits medidas. A magnetização $M$ é definida como a soma dos operadores de Pauli $Z$ de qubit único,
$$
M = \sum_{j=1}^N Z_j,
$$
onde $N$ é o número de qubits de sítio. Seu valor para uma cadeia de bits é igual à diferença entre o número de zeros e o número de uns. Medir o estado GHZ produz o estado todo zeros ou o estado todo uns com igual probabilidade, então a magnetização seria $+N$ metade do tempo e $-N$ a outra metade do tempo. Na presença de erros devido ao ruído, outros valores também apareceriam, mas se o ruído não for muito grande, a distribuição ainda seria concentrada perto de $+N$ e $-N$.

Para as cadeias de bits brutas antes da decodificação, a distribuição da magnetização seria equivalente à de cadeias de bits uniformemente aleatórias, na ausência de ruído.

A célula de código a seguir plota a magnetização das cadeias de bits brutas e das cadeias de bits decodificadas no ângulo $R_{ZZ}$ de $\frac{\pi}{2}$.